In [1]:
%matplotlib qt

import time
import copy
import numpy as np
from skimage.filters import gaussian

from tifffile import imread, imsave
import _pickle as pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d_radial import NetSurf3dRadial
from data4d_radial import Data4dRadial

# Load time-lapse data...

In [2]:
#filenames = [ '/home/wissmann/Cell_Model-0.0.1-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1).tif']
#filenames = [ '/Users/wissmann/BobSeg/Cell_croppedModel-0.0.1-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1).tif']
filenames = [ '/Users/wissmann/BobSeg/Vecad-flidsRed-48hpf.lif - fish1-48hpf-1.tif']
#    '/Users/micha/Desktop/Uni und so/CBG HiWi/Membrane Localization/Cell_Model-0.0.1-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1).tif']
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t1.TIF'] #,
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t2.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t3.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t4.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t5.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t6.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t7.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t8.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t9.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t10.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t11.TIF',
#             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t12.TIF']
#filename = '/Users/micha/Desktop/Uni und so/CBG HiWi/Downloads/MAX_LP320-06-08_Concat-51-100.tif'
#filename = '/Users/micha/Desktop/Uni und so/CBG HiWi/Membrane Localization/Cell_Model-0.0.1-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1).tif'
#filename = '/Users/micha/Desktop/Uni und so/CBG HiWi/Membrane Localization/Cell_Model-0.0.1(wave)-3D_Sim-PSF(0.2 0.2 0.6) PXS(0.1 0.1 0.1).tif'
#filename = '/Users/micha/Desktop/Uni und so/CBG HiWi/Membrane Localization/Cell_Model-0.0.1(wave)-3D_Sim-PSF(0.07 0.07 0.60) PXS(0.03 0.03 0.10).tif'

In [13]:
data = Data4dRadial( filenames, pixelsize=(1.,1.,1.), silent=False )
data.set_seg_params(K=80,max_delta_k=3)
objects = []
ecells = []

Dimensions of frame 0 :  (213, 738, 792)


Min radius and max radius at which to look for cells (around center points (seed points))

In [44]:
max_rs = (data.images[0].shape[2] / 2+20,
        data.images[0].shape[1] / 2+20,
        data.images[0].shape[0] / 2)

#max_rs=(600,600,100)

min_rs = (1,1,1)

## adding E cells

In [45]:
Ea = data.init_object("Ea")
objects.append(Ea)
ecells.append(Ea)

Ep = data.init_object("Ep")
objects.append(Ep)
ecells.append(Ep)

In [46]:
data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(200,200,100), frame_to=0, segment_it=True )
#data.track(Ea, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=3)

Added appearance for "Ea" in frame 0 with seed coordinates [200. 200. 100.]
      Optimum energy:  25403.0
      Nodes in/out:  15538 25422


In [47]:
data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(500,500,100), frame_to=0, segment_it=True )
# data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), segment_it=True )
# data.track(Ep, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ep" in frame 0 with seed coordinates [500. 500. 100.]
      Optimum energy:  23542.0
      Nodes in/out:  28849 12111


In [48]:
w = data.show_frame(0,show_surfaces=True,stackUnits=[1.,1.,1.],raise_window=True)

## adding direct neighbors

In [19]:
neighbors = []
for i in range(12):
    n = data.init_object("n"+str(i))
    objects.append(n)
    neighbors.append(n)

In [20]:
nseeds = [( 90,390,12),
          (150,410,12),
          (200,390,12),
          (190,315,12),
          (145,265,12),
          (110,230,12),
          ( 40,255,12),
          ( 45,300,18),
          ( 67,355,18),
          (110,275,24),
          (130,315,24),
          (125,360,24)]
for i in range(len(nseeds)):
    data.add_object_at( neighbors[i], min_rs, max_rs, frame=0, seed=nseeds[i], segment_it=True )
    data.track(neighbors[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "n0" in frame 0 with seed coordinates [ 90. 390.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "n1" in frame 0 with seed coordinates [150. 410.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "n2" in frame 0 with seed coordinates [200. 390.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "n3" in frame 0 with seed coordinates [190. 315.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "n4" in frame 0 with seed coordinates [145. 265.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "n5" in frame 0 with seed coordinates [110. 230.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "n6" in frame 0 with seed coordinates [ 40. 255.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "n7" in frame 0 with seed coordinates [ 45. 300.  18.]


## adding others

In [21]:
others = []
for i in range(7):
    o = data.init_object("o"+str(i))
    objects.append(o)
    others.append(o)

In [22]:
oseeds = [( 70,205,12),
          (145,170,12),
          ( 50,160,12),
          (105,140,12),
          (190,220,12),
          (200,265,12),
          (220,335,20)]
for i in range(len(oseeds)):
    data.add_object_at( others[i], min_rs, max_rs, frame=0, seed=oseeds[i], segment_it=True )
    data.track(others[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "o0" in frame 0 with seed coordinates [ 70. 205.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "o1" in frame 0 with seed coordinates [145. 170.  12.]
      Optimum energy:  433.0
      Nodes in/out:  15169 191
Added appearance for "o2" in frame 0 with seed coordinates [ 50. 160.  12.]
      Optimum energy:  4277.0
      Nodes in/out:  14652 708
Added appearance for "o3" in frame 0 with seed coordinates [105. 140.  12.]
      Optimum energy:  9882.0
      Nodes in/out:  12736 2624
Added appearance for "o4" in frame 0 with seed coordinates [190. 220.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "o5" in frame 0 with seed coordinates [200. 265.  12.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0
Added appearance for "o6" in frame 0 with seed coordinates [220. 335.  20.]
      Optimum energy:  0.0
      Nodes in/out:  15360 0


# Save / Load

In [ ]:
data.save('/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry.pkl')

In [ ]:
data = Data4dRadial( [], silent=False )
data.load('/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry.pkl')
objects = range(len(data.object_names))
ecells = [0,1]
neighbors = range(2,2+12)
others = range(2+12,2+12+7)

# Visualisation stuff 

In [12]:
w = data.show_frame(0,show_surfaces=True,stackUnits=[1.,1.,1.],raise_window=True)
# data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
# try:
#     data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
#     data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
# except:
#     None
#data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)

/Users/wissmann/py37/lib/python3.7/site-packages/pyopencl/cffi_cl.py:1521: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [27]:
for f in range(len(data.images)):
    print('Exporting frame ',f+1,'of',len(data.images))
    w = data.show_frame(f,stackUnits=[1.,1.,5.],raise_window=False)
    data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
    try:
        data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
        data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
    except:
        None
    data.save_current_visualization( '3dframes/frame'+str(f)+'.jpg' )

INFO:spimagine.gui.glwidget | saving frame as 3dframes/frame0.jpg


Exporting frame  1 of 1


In [14]:
data.object_volumes[Ea]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]